In [3]:
#https://www.pinecone.io/
#https://app.pinecone.io/organizations/-NgXykLJD3aeQmvj62TG/projects/6cbf827f-8f37-433f-b0ad-72747bb8e36d/indexes?sessionType=signup

In [4]:
!pip install langchain
!pip install pinecone-client==2.2.4
!pip install pypdf

In [5]:
!pip install sentence-transformers==2.2.2

In [6]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


In [7]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [8]:
data

[Document(page_content='YOLO-Z: Improving small object detection in YOLOv5 for\nautonomous vehicles\nAduen Benjumea* Izzeddin Teeti†Fabio Cuzzolin†Andrew Bradley*\n17065125@brookes.ac.uk, 19136994@brookes.ac.uk,\nfabio.cuzzolin@brookes.ac.uk, abradley@brookes.ac.uk\nAbstract\nAs autonomous vehicles and autonomous racing rise in popularity, so does the need for faster and more accurate\ndetectors. While our naked eyes are able to extract contextual information almost instantly, even from far away,\nimage resolution and computational resources limitations make detecting smaller objects (that is, objects that occupy\na small pixel area in the input image) a genuinely challenging task for machines and a wide-open research ﬁeld. This\nstudy explores how the popular YOLOv5 object detector can be modiﬁed to improve its performance in detecting\nsmaller objects, with a particular application in autonomous racing. To achieve this, we investigate how replacing\ncertain structural elements of the

In [9]:
len(data)

11

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [11]:
print("length of text chunks",len(text_chunks))

length of text chunks 93


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [13]:
query_result = embeddings.embed_query("Goa yay")

In [14]:
print(len(query_result))

384


In [15]:
import os
from google.colab import userdata
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [16]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "myindex" # put in the name of your pinecone index here


In [17]:
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [18]:
query = "What is yolo?"
docs = docsearch.similarity_search(query, k=3)
docs

[Document(page_content='Figure 7: Performance comparison between the YOLOv5 and YOLO-Z families of models, plotting mAP (top), mAP for small objects (middle)\nand mAP for medium objects (bottom) against inference time (ms). Clearly the superior average performance of YOLO-Z is achieved at the smaller\nscale, while performance is stable and very close to 1 at the medium scale.\n4.2 Inﬂuence of neck architecture'),
 Document(page_content='4.5 Other factors\nFigure 8: Visual demonstration of the improved detec-\ntion results of YOLO-Z S (bottom) compared to YOLOv5\nS (top) over a region of a sample image covering far away /\nsmall scale objects. Yellow and blue cone detections are\nshown as bounding boxes in the respective colours, de-\ntections missed by both models are shown as red boxes,\ndetections missed by YOLOv5 but correctly identiﬁed by\nYOLO-Z S as red circles. One can observe that the im-'),
 Document(page_content='With YOLOv1 [23], object detection is presented as a regression

In [19]:
!pip install langchain-google-genai

In [20]:
import google.generativeai as genai

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.5,convert_system_message_to_human=True)

In [23]:
retriever=docsearch.as_retriever()

In [35]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm,
                                  chain_type="stuff",
                                  retriever=retriever)

In [25]:
def process_llm_response(llm_response):
    print(llm_response['result'])

In [30]:
query = "what is yolov5"
llm_response = qa_chain(query)
process_llm_response(llm_response)

YOLOv5 is a real-time object detection model that was released shortly after YOLOv4. It is fully written in the PyTorch framework and has a slightly modified backbone to simplify it, which offers the potential to adapt to the system's requirements and opens the way for additional changes.


In [37]:
query = "what is YOLO-Z"
llm_response = qa_chain(query)
process_llm_response(llm_response)

YOLO-Z is a family of object detection models that were introduced in the paper "YOLO-Z: A Novel Architecture for Object Detection". The YOLO-Z models are based on the YOLOv5 architecture, but they incorporate several new features that improve their performance, including a new neck architecture, a new loss function, and a new training strategy.


In [39]:
query = "compare the performance between yolov5 and yolo-z"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Figure 7 shows a performance comparison between the YOLOv5 and YOLO-Z families of models, plotting mAP (top), mAP for small objects (middle), and mAP for medium objects (bottom) against inference time (ms). Clearly, the superior average performance of YOLO-Z is achieved at the smaller scale, while performance is stable and very close to 1 at the medium scale.

Table 2 compares performance and inference time of YOLOv5 and YOLO-Z (optimal values for each scale in bold). Note that the table only shows performance on the yellow and blue classes, as they are the best represented in the dataset according to Figure 1.


In [29]:

query = "what is yolov5"
print(qa_chain.invoke(query))

{'query': 'what is yolov5', 'result': 'YOLOv5 is a real-time object detection system that was released shortly after YOLOv4. It is fully written in the PyTorch framework and shares the same design as YOLOv4, but with a slightly modified backbone that offers the potential for adaptation to specific system requirements.'}


**Q&A Using Langchain**

In [50]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma


In [51]:
import google.generativeai as genai
llm = ChatGoogleGenerativeAI(model="gemini-pro")

#llm = (model="models/aqa",temperature=0.5,convert_system_message_to_human=True)

In [60]:
template = """
You are a helpful AI assistant.
Answer based on the context provided.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)


In [53]:
response=retrieval_chain.invoke({"input":"compare the performance between yolov5 and yolo-z"})

#Print the answer to the question
print(response["answer"])

Figure 7 shows the performance comparison between the YOLOv5 and YOLO-Z families of models. YOLO-Z consistently outperforms YOLOv5 in terms of mAP, mAP for small objects, and mAP for medium objects. This superior performance is particularly evident at the smaller scale, where YOLO-Z achieves a mAP of over 90%.


In [55]:
response=retrieval_chain.invoke({"input":"how performance of yolov5 can be modified."})

#Print the answer to the question
print(response["answer"])

The backbone of YOLOv5 can be modified to simplify it, which offers the potential to adapt to the system's requirements and opens the way for additional changes. If a single backbone element is modified, more drastic changes can be applied for additional effects.


In [56]:
response=retrieval_chain.invoke({"input":"what does this research paper talks about."})

#Print the answer to the question
print(response["answer"])


This research paper focuses on improving the performance of object detection systems in autonomous vehicles, particularly for small objects like cones, which are critical for safe navigation. The paper explores various approaches to enhance detection accuracy and reduce inference time, ultimately aiming to improve the overall safety and efficiency of autonomous vehicles.
